## Jupyter Notebook zur Umwandlung von Telegram Exportdateien (JSON) in XML und xlsx

Autor: Simon Meier-Vieracker (https://github.com/fussballlinguist/)

In [1]:
import json
import re
import xml.etree.ElementTree as et
from xml.dom import minidom
import pandas as pd

In [2]:
with open("Data/telegram_anonymized.json") as f:
    txt = f.read()
    txtj = json.loads(txt)

In [3]:
# XML anlegen
corpus = et.Element("corpus")

# Listen für Dataframe anlegen
cols = ["id","date","from","reply_to","text"]
rows = []

for message in txtj["messages"]:
    
    #Reine Sytemmeldungen übergehen
    if message["type"] != "message":
        continue
    
    #Metadaten auslesen
    msg_id = str(message["id"])
    msg_date = str(message["date"])
    msg_from = message["from"]
    if msg_from == None:
        msg_from = "nA"
    if "reply_to_message_id" in message:
        msg_reply = str(message["reply_to_message_id"])
    else:
        msg_reply = "nA"
    
    #XML Element anlegen und Metadaten als Attribute schreiben
    message_element = et.SubElement(corpus, "message", {"id":msg_id,"date":msg_date,"from":msg_from, "reply_to":msg_reply})
    
    #Nachrichtentext auslesen
    msg_content = message["text"]
    
    #Nachrichten ohne Text übergehen
    if len(msg_content) == 0:
        continue
        
    #Falls die Nachrichten aus mehreren Textblöcken bestehen oder Links enthalten,
    #werden sie als Liste gefasst. Diese wird zu einem String ohne Zeilenumbrüche zusammengefügt.
    if type(msg_content) == list:
        txt_content = ""
        for part in msg_content:
            if type(part) == str:
                txt_content += part                
    elif type(msg_content) == str:
        txt_content = msg_content
                        
    msg_content = txt_content
    msg_content = msg_content.replace("\n", " ")
    msg_content = re.sub(r'\s+',' ',msg_content)
    
    #Text in XML schreiben
    message_element.text = msg_content
    
    # Listen befüllen
    rows.append({"id":msg_id,
                 "date":msg_date,
                 "from":msg_from,
                 "reply_to":msg_reply,
                 "text":msg_content})
    

In [4]:
xmlstr = minidom.parseString(et.tostring(corpus)).toprettyxml(indent="  ")
print(xmlstr)
with open("Data/telegram_anonymized.xml", "w") as xmlout:
    xmlout.write(xmlstr)

<?xml version="1.0" ?>
<corpus>
  <message id="754219" date="2024-06-10T08:02:38" from="beba99fa4b9802c940bf8f5257843872b6343029728a6e42e3eb2261d70f42b8" reply_to="754216">Stark stark stark 💪</message>
  <message id="754220" date="2024-06-10T10:08:42" from="9ae4bc0e32db0e3484cd398459d20f9b4f79cce36667428181bf037131a3c987" reply_to="nA">BTW Wer den Research von Twitch abgeholt hat kann bis dieser abläuft jedes Wablu zu Altaria entwickeln und Mondgewalt bekommen. Auch wenn man den Research schon abgeschlossen hat, gerade getestet.</message>
  <message id="754221" date="2024-06-10T10:25:33" from="c60ca86cf069abd0b6e02461b1be5a7282318f2fb81dc1578e36f532824e2fe5" reply_to="754220">Ach mist ich hab mein hundo schon entwickelt y.y</message>
  <message id="754222" date="2024-06-10T10:30:29" from="1fd19561eec14cd31fa6c76d1a99af7d92b2986206f2191bf2dda3950cf79e0a" reply_to="754220">Man muss aber sein twitch acc mit pogo verknüpfen right?</message>
  <message id="754223" date="2024-06-10T10:30:40"

In [5]:
# DataFrame anlegen und als Excel (oder CSV) speichern 
df = pd.DataFrame(rows, columns=cols)
df.to_excel("Data/telegram_anonymized.xlsx")